In [1]:
import pandas as pd
import plotly.express as px
import json
from pandas.api.types import CategoricalDtype

with open('sources.json') as input_file:
    sources = json.load(input_file)
    
with open('channels.json') as input_file:
    channels = json.load(input_file)

In [2]:
source_df = pd.DataFrame()
source_df['backend'] = [x['backend'] for x in sources]
source_df['stream_address'] = [x['stream'] for x in sources]

source_df['stream_port'] = source_df['stream_address'].apply(lambda x: x.split(':')[-1])
source_df['source_name'] = source_df['stream_address'].apply(lambda x: x.split(':')[1][2:])

source_df['location'] = source_df['source_name'].apply(lambda x: x.split('-')[0])
source_df['host_type'] = source_df[source_df['source_name'].apply(lambda x: len(x.split('-'))) > 1]['source_name'].apply(lambda x: x.split('-')[1][0:4])
source_df['device'] = source_df[source_df['source_name'].apply(lambda x: len(x.split('-'))) > 2]['source_name'].apply(lambda x: x.split('-')[2])

source_df['n_connections_per_source'] = source_df.groupby("source_name")['stream_port'].transform('count')

px.histogram(source_df, x='n_connections_per_source',
            title='Number of connection to a single host (different port)')

In [3]:
channel_df = pd.DataFrame()
channel_df['channel_name'] = [x['name'] for x in channels[0]['channels']]
channel_df['stream_address'] = [x['source'] for x in channels[0]['channels']]
channel_df['backend'] = [x['backend'] for x in channels[0]['channels']]
channel_df['shape'] = [x['shape'][0] for x in channels[0]['channels']]
channel_df['dtype'] = [x['type'] for x in channels[0]['channels']]
channel_df['modulo'] = [x['modulo'] for x in channels[0]['channels']]

channel_df['location'] = channel_df['channel_name'].apply(lambda x: x.split('-')[0])
channel_df['host_type'] = channel_df['channel_name'].apply(lambda x: x.split('-')[1].split(':')[0][0:4])

channel_df['dtype_bytes_per_element'] = channel_df['dtype'].map({
    'float64': 8, 'uint64': 8,
    'float32': 4, 'uint32': 4, 'int32': 4,
    'uint16': 2, 'int16': 2,
    'int8': 1, 'bool': 1, 'string': 1
})
channel_df['bytes_per_pulse'] = channel_df['dtype_bytes_per_element'] * channel_df['shape']
channel_df['bytes_per_second'] = channel_df['dtype_bytes_per_element'] * channel_df['shape'] * 100 / channel_df['modulo']

source_category_order = ["string","16bit scalar","32bit scalar","64bit scalar","16bit 2048 array (mod 100)","32bit 16 array","16bit 64 array","8bit 176 array",
                         "bool 256 array","16bit 2048 array (mod 10)","16bit 512 array","16bit 1024 array","16bit 2048 array","64bit 659 array","32bit 2048 array",
                         "32bit 2560 array","32bit 1748 array","64bit 2000 array", '16bit 20000 array']
source_category = {
    100: source_category_order[0],
    200: source_category_order[1],
    400: source_category_order[2],
    800: source_category_order[3],
    4096: source_category_order[4],
    6400: source_category_order[5],
    12800: source_category_order[6],
    17600: source_category_order[7],
    25600: source_category_order[8],
    40960: source_category_order[9],
    102400: source_category_order[10],
    204800: source_category_order[11],
    409600: source_category_order[12],
    527200: source_category_order[13],
    819200: source_category_order[14],
    1024000: source_category_order[15],
    1398400: source_category_order[16],
    1600000: source_category_order[17],
    4000000: source_category_order[18]
}

    
channel_df["source_category"] = channel_df['bytes_per_second'].map(source_category).fillna('Scalar modulo 100')
channel_df['source_category'] = channel_df['source_category'].astype(CategoricalDtype(categories=['Scalar modulo 100'] + source_category_order, ordered=True))
    
px.histogram(channel_df, x='source_category',
            title="Number of channels in each source category").update_xaxes(categoryorder='total descending')

In [4]:
px.bar(channel_df.groupby(['source_category'])['bytes_per_second'].sum().reset_index().sort_values(by='bytes_per_second', ascending=False), x='source_category', y='bytes_per_second',
       title='Source category contribution to total throughput per second')

In [24]:
merged_df = source_df.merge(channel_df, how='outer', left_on='stream_address', right_on='stream_address', suffixes=('_s', '_c')).dropna()
merged_df['n_channels_per_source'] = merged_df.groupby('stream_address')['channel_name'].transform('count')

px.histogram(merged_df, x='n_channels_per_source', title='Number of channels per source', color='source_category')

In [23]:

merged_df[merged_df['source_category'].isna()]

backend_s                   stream_address stream_port  \
58     sf-databuffer         tcp://sf-daqsync-05:9001        9001   
248    sf-databuffer     tcp://S30-CVME-DBPM4103:9000        9000   
323    sf-databuffer   tcp://SATMA02-CVME-LLRF2:20000       20000   
497    sf-databuffer  tcp://S10BC01-CVME-DBLM184:9999        9999   
1287   sf-databuffer         tcp://sf-sioc-cs-81:8105        8105   
1497   sf-databuffer     tcp://SARES11-CPCL-GES1:9980        9980   
2268   sf-databuffer         tcp://sf-daqsync-02:9005        9005   
3152   sf-databuffer         tcp://sf-daqsync-05:9003        9003   
3324   sf-databuffer               tcp://camtest:9999        9999   
3421   sf-databuffer     tcp://SARES21-CPCL-GES1:9970        9970   
3513   sf-databuffer         tcp://sf-daqsync-06:9003        9003   
3578   sf-databuffer     tcp://SARES21-CPCL-GES1:9980        9980   
4166   sf-databuffer  tcp://SAROP11-CPCL-PALM118:9990        9990   
4408   sf-databuffer  tcp://sf-daqsync-04.psi.ch:9003        9003   
4549   sf-databuffer     tcp://SFTEST-CVME-DBPM1:9000        9000   
4605   sf-databuffer      tcp://SIN-CVME-LAS0041:9999        9999   
4622   sf-databuffer  tcp://sf-daqsync-06.psi.ch:9015        9015   
4932   sf-databuffer         tcp://sf-daqsync-06:9001        9001   
5104   sf-databuffer  tcp://SATBD02-CVME-DICT499:9999        9999   
5129   sf-databuffer    tcp://SAROP21-CVME-PBPS1:9999        9999   
5384   sf-databuffer  tcp://sf-daqsync-06.psi.ch:9018        9018   
5527   sf-databuffer    tcp://SAROP21-CVME-PBPS3:9999        9999   
5528   sf-databuffer  tcp://sf-daqsync-03.psi.ch:9003        9003   
5549   sf-databuffer  tcp://sf-daqsync-06.psi.ch:9007        9007   
5773   sf-databuffer  tcp://sf-daqsync-03.psi.ch:9004        9004   
5857   sf-databuffer   tcp://SATMA02-CVME-LLRF1:20002       20002   
6053   sf-databuffer   tcp://SATMA02-CVME-LLRF2:20002       20002   
6080   sf-databuffer  tcp://sf-daqsync-09.psi.ch:9000        9000   
6082   sf-databuffer    tcp://SAROP21-CVME-PBPS2:9999        9999   
6271   sf-databuffer  tcp://sf-daqsync-04.psi.ch:9001        9001   
6272   sf-databuffer      tcp://SAR-CVME-TIFALL6:9999        9999   
6362   sf-databuffer  tcp://sf-daqsync-06.psi.ch:9014        9014   
6500   sf-databuffer         tcp://sf-daqsync-05:9011        9011   
7273   sf-databuffer  tcp://sf-daqsync-03.psi.ch:9001        9001   
7425   sf-databuffer  tcp://sf-daqsync-06.psi.ch:9017        9017   
7510   sf-databuffer     tcp://SARES12-CPCL-GES1:9999        9999   
7802   sf-databuffer  tcp://sf-daqsync-03.psi.ch:9011        9011   
7934   sf-databuffer     tcp://sates21-cpcl-ges1:9989        9989   
8221   sf-databuffer         tcp://sf-daqsync-10:9001        9001   
8482   sf-databuffer  tcp://sf-daqsync-04.psi.ch:9011        9011   
8623   sf-databuffer     tcp://SARES11-CPCL-GES1:9970        9970   
8976   sf-databuffer   tcp://SATMA02-CVME-LLRF1:20000       20000   
9152   sf-databuffer     tcp://SARES21-CPCL-GES1:9990        9990   
9688   sf-databuffer   tcp://SFTEST-CVME-CON1331:9999        9999   
10527  sf-databuffer      tcp://S30CB08-CVME-ILK:9000        9000   
10664  sf-databuffer     tcp://SARES21-CPCL-GES1:9999        9999   
11304  sf-databuffer     tcp://SARES11-CPCL-GES1:9990        9990   

                source_name location_s host_type_s     device  \
58            sf-daqsync-05         sf        daqs         05   
248       S30-CVME-DBPM4103        S30        CVME   DBPM4103   
323      SATMA02-CVME-LLRF2    SATMA02        CVME      LLRF2   
497    S10BC01-CVME-DBLM184    S10BC01        CVME    DBLM184   
1287          sf-sioc-cs-81         sf        sioc         cs   
1497      SARES11-CPCL-GES1    SARES11        CPCL       GES1   
2268          sf-daqsync-02         sf        daqs         02   
3152          sf-daqsync-05         sf        daqs         05   
3324                camtest    camtest         NaN        NaN   
3421      SARES21-CPCL-GES1    SARES21        CP

In [ ]:
channel_df[channel_df['channel_name']=='SARUN20-DWSC010:CURR_CYCL']